## 症状补充  
* 最初的症状列表是通过二级部位爬取的  
* 发现二级部位爬取的症状不全，本文的第一部分改用一级部位爬取  
* 发现疾病的典型症状还有不在症状库中的，本文第二部分补充爬取疾病的典型症状

In [4]:
import sys
sys.path.append('/home/ian/code/github')

### 第一部分改用一级部位爬症状

In [2]:
from utils.pickle_wrapper import pickle_wrapper as pkw

In [3]:
all_zz_list1 = pkw.loadfromfile('一级部位zz列表.pkl')
all_zz_list1[:2]

[('皮肤干燥', 'http://jbk.39.net/zhengzhuang/pfgz/', '全身'),
 ('结肠血管发育不良', 'http://jbk.39.net/zhengzhuang/jcxgfybl/', '腹部')]

In [4]:
from utils.pymongo_wrapper import pymongo_wrapper as pmw
client = pmw()
table = client.get_collection('jiankang39','zz')

In [12]:
from utils.crawl import requests_wrapper as rw
from utils.crawl import pyquery_wrapper as pw

In [10]:
def parse_zz(url):
    '''
    解析症状
    '''
    r1 = rw.requests_wrapper(url)
    html = r1.get_text(encoding='gbk')
    d = pw.pyquery_wrapper(html).Root
    dict1 = dict()
    dict1['url'] = url
    name = d('.list_tit > div.tik.clearfix > a > h1').eq(0).text().strip()
    dict1['症状名称'] = name
    alias = d('.list_tit > div.tik.clearfix > .tik.alias').eq(0).text().strip()
    if alias:
        alias = alias[:-1].split('：')[1]
        dict1['别名'] = alias
    dict1['简介'] = d('#intro > p').eq(0).text()
    #症状起因
    r2 = rw.requests_wrapper('{}zzqy/'.format(url))
    d2 = pw.pyquery_wrapper(r2.get_text(encoding='gbk')).Root
    t = d2('div.item.catalogItem > p').eq(0).text().strip()
    if t:
        dict1['症状起因'] = t
    #相关疾病relateDis
    ills = d2('#relateDis tr > td.name > a')
    l = list()
#     l2 = list()
    for i in range(len(ills)):
        l.append(ills.eq(i).attr.title)
#         l2.append((ills.eq(i).attr.title,ills.eq(i).attr.href,d2('#relateDis tr > td:nth-child(2)').eq(0).text().strip().split()))
    dict1['相关疾病'] = l
    return dict1

In [17]:
parse_zz('http://jbk.39.net/zhengzhuang/pfgz/')

{'url': 'http://jbk.39.net/zhengzhuang/pfgz/',
 '症状名称': '皮肤干燥',
 '简介': '皮肤干燥是指因季节变化，缺水和贫血等原因，使得皮肤变厚、变粗糙。秋冬季节，人体的皮脂、水分分泌会逐渐减少，皮肤明显变得干燥，称为干性皮肤，尤其是中老年人因水分大量减少，皮肤表层会显得更粗燥，会常看见他们的手脚四肢、小腿处会有干裂、发痒的情形，在无法忍受干痒，会不断的去抓痒，造成皮肤有伤口，引起发炎或流脓。',
 '症状起因': '因季节变化，缺水和贫血等原因，使得皮肤变厚、变粗糙。在临床皮肤含水量测试中，正常表皮含水量应是25%，但如今都市女性肌肤含水量已降至15%左右。',
 '相关疾病': ['皲裂性湿疹',
  '慢性湿疹',
  '皮肤病',
  '过敏性眼睑皮肤炎',
  '夏季热',
  '凉燥',
  '秋燥',
  '皲裂疮',
  '甲状腺功能减低所致贫血',
  '原发性直立性低血压',
  '甲状腺炎',
  '糖尿病足',
  '铁利用性贫血',
  '手足皲裂',
  '鱼鳞病',
  '夜盲症',
  '足月小样儿',
  '手足癣',
  '汽油中毒',
  '营养不良',
  '血栓闭塞性脉管炎',
  '血色病',
  '交感神经链综合征',
  '阿尔采末病',
  '成人全垂体功能减退',
  '玫瑰糠疹',
  '神经性皮炎',
  '干燥综合征',
  '绝经期关节炎',
  '维生素A缺乏神经病',
  '小儿尿崩症',
  '肾病综合症',
  '小儿营养不良',
  '贫血',
  '羊水过少',
  '无汗症',
  '皮脂缺乏症',
  '小儿维生素A缺乏病',
  '尿崩症',
  '小儿营养不良性水肿',
  '冬季瘙痒症',
  '玻璃纤维引起的皮肤病',
  '无机碱皮肤损伤',
  '甲状旁腺功能减退症',
  '亚临床甲状腺功能减退症',
  '老年甲状腺功能减退症',
  '生长激素缺乏症',
  '希恩综合征',
  '神经性厌食症与神经性贪食症',
  '肥胖性生殖无能综合征']}

In [18]:
%%time
#爬取zz详情
errorlist = list()
for t in all_zz_list1:
    if list(client.find_all(table,{'url':t[1]})):
        continue
    try:
        d = parse_zz(t[1])
        d['c1'] = t[2]
        table.insert_one(d)
        print('{} is inserted!'.format(t[0]))
    except Exception as e:
        print(e)
        print(t[1])
        errorlist.append(t[1])

print(len(errorlist))
if errorlist:
    pickle.dump2file(errorlist, 'zzerrorlist.pkl')

喜凉怕热 is inserted!
肋间隙变窄 is inserted!
阴道持续流血不停 is inserted!
桶状胸 is inserted!
胰腺内钙化 is inserted!
隐性产后出血 is inserted!
胸腔出口综合征 is inserted!
肺动脉闭锁 is inserted!
尾骶部疼痛 is inserted!
胃危象 is inserted!
胃酸减少 is inserted!
残株型骨折 is inserted!
气血失和 is inserted!
产后消化系统症状 is inserted!
胸闷 is inserted!
胃窦动力低下 is inserted!
手部功能完全丧失 is inserted!
甲身凹沟 is inserted!
风寒湿痹 is inserted!
CO2潴留 is inserted!
横结肠移位 is inserted!
十二指肠高度扩张 is inserted!
阑尾组织纤维化 is inserted!
虚火 is inserted!
两肺弥漫或散在可逆性哮鸣... is inserted!
组织液渗出 is inserted!
湿性脚气 is inserted!
脉压增宽 is inserted!
肝气犯脾 is inserted!
腹壁反射减弱或消失 is inserted!
浅筋膜炎 is inserted!
抽筋 is inserted!
脾气虚 is inserted!
骨骼肿块 is inserted!
痰火上壅清窍 is inserted!
��音 is inserted!
胎势异常 is inserted!
胃纳减退 is inserted!
毛发脱落 is inserted!
韧带断裂 is inserted!
细颗粒管型 is inserted!
胸壁塌陷 is inserted!
下腹部反跳痛 is inserted!
乙肝表面抗原（HBsAg）阳 is inserted!
五心烦热 is inserted!
梅毒感染 is inserted!
肺通气一血流比例失调 is inserted!
胸部有过度回响 is inserted!
惧站立，喜依托 is inserted!
关节响痛 is inserted!
血虚眩运 is inserted!
胆囊增大 is inserte

拇指三节 is inserted!
上腹部压痛 is inserted!
前胸压迫性疼痛 is inserted!
尿渗透压下降 is inserted!
脐血 is inserted!
乳房内光滑质韧的球形肿块 is inserted!
脐腹部或胁肋部患有癖块 is inserted!
肝火 is inserted!
扣痛 is inserted!
脾栓塞 is inserted!
运动气喘 is inserted!
排气障碍 is inserted!
小腹冷痛 is inserted!
颈部淋巴结肿大 is inserted!
生殖器外伤 is inserted!
脐带血循环障碍 is inserted!
腹部肥满 is inserted!
四支怠惰 is inserted!
食管左壁形成压足�E is inserted!
癌性输尿管狭窄 is inserted!
骨龄延迟 is inserted!
小便淋漓 is inserted!
红细胞畸形 is inserted!
肺阴虚 is inserted!
创伤性宫腔粘连 is inserted!
小指半月甲呈粉红色 is inserted!
劳力性晕厥 is inserted!
腹肌紧张 is inserted!
输尿管扩张 is inserted!
闭经-乳溢-不育三联征 is inserted!
肝被膜紧张 is inserted!
肾异位伴随输尿管异位 is inserted!
肝细胞索支架塌陷 is inserted!
气结 is inserted!
腰痛卧床加重 is inserted!
阻塞性黄疸 is inserted!
阴囊肿大 is inserted!
输尿管痉挛 is inserted!
肝癖 is inserted!
韧带扭伤 is inserted!
男子性功能障碍 is inserted!
肝积水 is inserted!
颈短 is inserted!
胎寒 is inserted!
经期前后小腹坠痛 is inserted!
胞宫纳精无力 is inserted!
频繁性惊醒 is inserted!
尿道口粘连 is inserted!
右下腹压痛 is inserted!
生理性腹泻 is inserted!
压缩型骨折 is inserted!
腰部钝痛 is inserte

侧肋骨痛 is inserted!
脊髓前角病变 is inserted!
心肌脓肿 is inserted!
小肠内的低pH is inserted!
气急 is inserted!
婴儿褐色或暗绿色的粘液便 is inserted!
胃扩张 is inserted!
婴儿睡觉惊跳 is inserted!
椎间隙变化 is inserted!
新生儿便秘 is inserted!
呼吸气为蒜臭味 is inserted!
产后发热 is inserted!
上半身代偿性多汗症 is inserted!
空腹时腹部有振水声者 is inserted!
钙化灶 is inserted!
痛性阴茎勃起 is inserted!
溃疡 is inserted!
乏力 is inserted!
支气管狭窄 is inserted!
胸廓畸形 is inserted!
等渗尿 is inserted!
子宫附件肿块 is inserted!
慢性疼痛 is inserted!
习惯性流产 is inserted!
淋巴道转移 is inserted!
下腹压痛 is inserted!
热结节 is inserted!
食欲缺乏 is inserted!
性发育停顿 is inserted!
暴力作用后腹痛 is inserted!
冷痛 is inserted!
腹部逐渐胀大 is inserted!
胸腹部畸形 is inserted!
溺多而频 is inserted!
凉结节 is inserted!
胎儿宫内骨折 is inserted!
长骨骨骺端钙化不全 is inserted!
嗜睡 is inserted!
行为紊乱奇特 is inserted!
不能向尺侧屈腕及屈环小指... is inserted!
室间隔缺损 is inserted!
肠壁坏死 is inserted!
腕下垂 is inserted!
肋膜腔积水 is inserted!
运动或情绪激动诱发的胸痛 is inserted!
胸骨压痛 is inserted!
肠道转运障碍 is inserted!
定向力障碍 is inserted!
排卵障碍 is inserted!
胃移位 is inserted!
宗筋失养 is inserted!
剧烈眩晕，站立不稳 is inserted

脘痞 is inserted!
胎动改变 is inserted!
隔膜孔道狭小 is inserted!
头颈部活动受限 is inserted!
尿道流出分泌物 is inserted!
醛固酮分泌增多 is inserted!
脓血性白带 is inserted!
锥体束病损 is inserted!
恶性胸水 is inserted!
小腹刺痛 is inserted!
月经血块过多 is inserted!
甲状腺扪及光滑坚实的椭圆... is inserted!
阳虚体质 is inserted!
弥漫性骨膜增厚 is inserted!
绝经后子宫内膜增厚 is inserted!
突发性绞痛 is inserted!
老年男性乳房无痛性肿块 is inserted!
胃痉挛 is inserted!
上肢周围性瘫痪 is inserted!
会阴部撕裂感 is inserted!
纤毛结构缺陷 is inserted!
胰腺功能不足 is inserted!
气体弥散障碍 is inserted!
双肺的吸气爆裂音 is inserted!
精子活力低 is inserted!
关节积液 is inserted!
胃丢失蛋白 is inserted!
精囊增大 is inserted!
头昏 is inserted!
耻骨直肠肌痉挛性肥大 is inserted!
喘鸣 is inserted!
柠檬色痰 is inserted!
肺血流量很多 is inserted!
休克较久后的肺部病变 is inserted!
疲劳 is inserted!
椎间盘退行性变 is inserted!
慢性腹痛 is inserted!
腰疼 is inserted!
混合型酸碱平衡紊乱 is inserted!
小便不易排空 is inserted!
肾性尿崩 is inserted!
前额头痛 is inserted!
不全流产 is inserted!
胸痛 is inserted!
一咳嗽胸口就疼 is inserted!
幽门狭窄 is inserted!
伸指无力 is inserted!
乙肝e抗原（HBeAg）阳性 is inserted!
进食样自动症 is inserted!
肾无功能 is inserted!
脐尿管残余 is insert

关节痛 is inserted!
胃灼热 is inserted!
老人剧烈咳嗽 is inserted!
疝区的疼痛或坠胀感 is inserted!
胸腹呈矛盾运动 is inserted!
绝经过渡期及绝经后闭经 is inserted!
脊肋角膨隆 is inserted!
阴道排出碎块组织 is inserted!
嗳气 is inserted!
假日心脏综合征 is inserted!
泻药性便秘 is inserted!
心包转移 is inserted!
胸骨后疼痛 is inserted!
肾部积水现象 is inserted!
不良建筑综合征 is inserted!
第5指短小弯曲 is inserted!
绝经后白带增多 is inserted!
阳明热盛 is inserted!
火热熏灼 is inserted!
容易发生青肿 is inserted!
乳腺外上象限腺体增厚 is inserted!
痰火郁结 is inserted!
肺部��音 is inserted!
腹股沟疼痛 is inserted!
脂肪泻 is inserted!
胸廓异常 is inserted!
黏液尿 is inserted!
输尿管梗阻 is inserted!
精液稠 is inserted!
浊气壅滞 is inserted!
阴囊瘙痒 is inserted!
婴儿缺钙 is inserted!
附睾肿大发硬 is inserted!
毛发干枯 is inserted!
阴道粘连甚至闭锁 is inserted!
上尿路结构和功能损害 is inserted!
胸部大片斑块 is inserted!
邪陷厥阴 is inserted!
肾乳头及肾盂出血 is inserted!
肝细胞内脂肪堆积过多 is inserted!
包皮垢 is inserted!
阴囊坠胀痛 is inserted!
阴部皮肤潮红 is inserted!
右上腹痛 is inserted!
腰背部疼痛 is inserted!
两肺毛玻璃样粟粒样或结节... is inserted!
心电图异常 is inserted!
重度烧伤 is inserted!
中上腹持续性疼痛 is inserted!
回盲部结核 is inserted!
肾间质水肿 is inser

咳出带臭味的干痂 is inserted!
胸部下垂 is inserted!
呛咳 is inserted!
肺癌胸水 is inserted!
老人便秘 is inserted!
上腹弥漫性压痛 is inserted!
胆汁潴留 is inserted!
内热哺热 is inserted!
筋膜疼痛 is inserted!
饭后心悸 is inserted!
发育性骨盆异常 is inserted!
腹中积癖 is inserted!
肝细胞坏死 is inserted!
精液带血 is inserted!
肚脐疼 is inserted!
胸椎腰化 is inserted!
胃粘膜脱垂 is inserted!
鞘膜积水 is inserted!
交感神经代偿性亢进 is inserted!
尿道口疼 is inserted!
消瘦严重者呈”皮包骨头”... is inserted!
酒精性震颤 is inserted!
婴儿体重增长缓慢 is inserted!
胸椎疼痛 is inserted!
强迫仰卧位 is inserted!
创伤 is inserted!
角弓反张位 is inserted!
肺气压伤 is inserted!
腹壁红斑 is inserted!
轻度烧伤 is inserted!
身体“福态” is inserted!
活动后气促 is inserted!
骶髂部剧烈疼痛 is inserted!
眩闷口眼偏斜 is inserted!
妊娠时持续性腹痛或腰酸腰... is inserted!
疋氏（Pickweckian）综合 is inserted!
心脏震颤 is inserted!
入球小动脉玻璃样变 is inserted!
运动性疲劳 is inserted!
心脏猝死 is inserted!
腹腔积血 is inserted!
腹部肿块 is inserted!
心功能不全 is inserted!
关节扭伤 is inserted!
精液少 is inserted!
孕妇腹胀 is inserted!
前臂和腕部的疼痛 is inserted!
胃脘饱胀 is inserted!
体重减轻 is inserted!
骨外团块 is inserted!
腰大肌脓肿 is inserted!
月经发黑 is i

宫缩乏力 is inserted!
刺痛 is inserted!
开睑及闭睑失用 is inserted!
膀胱排空不全 is inserted!
头重如裹 is inserted!
肝热及胆 is inserted!
新生儿斜颈 is inserted!
心内膜弹力纤维增生 is inserted!
突发性疼痛 is inserted!
颈椎分节异常 is inserted!
内分泌失调 is inserted!
逆行性细菌感染 is inserted!
心音微弱 is inserted!
双手轮替动作笨拙 is inserted!
肾小球内“三高” is inserted!
内出血 is inserted!
高原反应 is inserted!
脐窝湿润有臭味 is inserted!
前斜角肌肥厚痉挛 is inserted!
肝静脉回流障碍 is inserted!
胆汁返流 is inserted!
尿锰粪锰 is inserted!
胸膜转移 is inserted!
屈性截瘫 is inserted!
纵隔增宽 is inserted!
继发性不孕 is inserted!
气机不畅 is inserted!
腕部疼痛 is inserted!
恶臭便 is inserted!
腰椎不稳 is inserted!
髓核脱水 is inserted!
反复肺炎 is inserted!
痔核 is inserted!
排尿过程中发生晕厥 is inserted!
前列腺分泌物含有许多脓缩... is inserted!
青春期胸部发育缓慢 is inserted!
感觉迟钝 is inserted!
颈椎肥大 is inserted!
脐周脓性分泌物较多 is inserted!
拇指半月甲呈粉红色时 is inserted!
精血不足 is inserted!
颈髓脱髓鞘性病变 is inserted!
涨奶 is inserted!
反射性腰痛 is inserted!
肠痉挛 is inserted!
内脏痛 is inserted!
痢下赤白粘冻 is inserted!
长骨粗大 is inserted!
下垂肩 is inserted!
厥心痛 is inserted!
结肠梗阻 is inserted!
男性小便刺痛 is inserted

NameError: name 'pickle' is not defined

In [22]:
l1 = list(filter(lambda t:t[1] in errorlist,all_zz_list1))
l1

[('冠状动脉供血不足', 'http://jbk.39.net/zhengzhuang/gzdmgxbz/', '胸部'),
 ('椎节分裂', 'http://jbk.39.net/zhengzhuang/zjfl/', '骨'),
 ('妊娠期胸痛', 'http://jbk.39.net/zhengzhuang/rsqxt/', '胸部'),
 ('乳腺内无痛单发的硬肿块', 'http://jbk.39.net/zhengzhuang/rxnwtdfdyzk/', '胸部'),
 ('血丝痰', 'http://jbk.39.net/zhengzhuang/xst/', '胸部'),
 ('恶心与呕吐', 'http://jbk.39.net/zhengzhuang/exyot/', '腹部'),
 ('颈椎间盘退变', 'http://jbk.39.net/zhengzhuang/jzjptb/', '骨'),
 ('脾失健运', 'http://jbk.39.net/zhengzhuang/psjy/', '腹部'),
 ('肝内胆管结石', 'http://jbk.39.net/zhengzhuang/gndgjs/', '腹部'),
 ('难免流产或不可避免流产', 'http://jbk.39.net/zhengzhuang/nmlchbkbmlc/', '女性生殖'),
 ('颈椎间盘退变', 'http://jbk.39.net/zhengzhuang/jzjptb/', '腰部')]

In [23]:
%%time
#爬取zz详情
errorlist1 = list()
for t in l1:
    if list(client.find_all(table,{'url':t})):
        continue
    try:
        d = parse_zz(t[1])
        d['c1'] = t[2]
        table.insert_one(d)
        print('{} is inserted!'.format(t[0]))
    except Exception as e:
        print(e)
        print(t[1])
        errorlist1.append(t[1])

冠状动脉供血不足 is inserted!
椎节分裂 is inserted!
妊娠期胸痛 is inserted!
乳腺内无痛单发的硬肿块 is inserted!
血丝痰 is inserted!
恶心与呕吐 is inserted!
E11000 duplicate key error collection: jiankang39.zz index: url_1 dup key: { : "http://jbk.39.net/zhengzhuang/jzjptb/" }
http://jbk.39.net/zhengzhuang/jzjptb/
脾失健运 is inserted!
肝内胆管结石 is inserted!
难免流产或不可避免流产 is inserted!
E11000 duplicate key error collection: jiankang39.zz index: url_1 dup key: { : "http://jbk.39.net/zhengzhuang/jzjptb/" }
http://jbk.39.net/zhengzhuang/jzjptb/
CPU times: user 345 ms, sys: 20.1 ms, total: 365 ms
Wall time: 2.9 s


In [34]:
ll = list(client.find_all(table,{'c2':{'$exists':False}},['症状名称','c1']))

In [35]:
len(ll)

2867

In [36]:
ll[:1]

[{'症状名称': '喜凉怕热', 'c1': '全身'}]

In [38]:
from utils.pandas_wrapper import pandas_wrapper as pdw

In [39]:
df = pdw.build_df_with_dict({'症状名称':[i['症状名称'] for i in ll],'c1':[ii['c1'] for ii in ll]})
df.head()

,症状名称,c1
0,喜凉怕热,全身
1,肋间隙变窄,胸部
2,阴道持续流血不停,女性生殖
3,桶状胸,胸部
4,胰腺内钙化,腹部


In [41]:
df.sort_values(by='c1').to_csv('zz1.csv',encoding='gbk')

### 第二部分补充爬取疾病的典型症状

In [6]:
from utils.pymongo_wrapper import pymongo_wrapper as pmw
pmw = pmw()
zztable = pmw.get_collection('jiankang39','zz')
distable = pmw.get_collection('jiankang39','diseases')

##### 获取典型症状列表

In [1]:
import requests
from pyquery import PyQuery as pq

In [2]:
def parsezzurls(url):
    '''
    解析疾病页面的症状urls
    '''
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36 Maxthon/5.2.1.6000'}
    html = requests.get(url,headers=headers)
    html.encoding = 'gbk'
    d = pq(html.text)
    dds = d('div.info > ul > li')
    for i in range(len(dds)):
        label = dds.eq(i)('i').eq(0).text()
        s = dds.eq(i).text()    
        if s.endswith('[详细]'):
            s = s[:-4].strip()
        ss = [i.strip() for i in s.split('：')]
        content = dds.eq(i)('a')
        if content and ss[0] in ['典型症状']:
            ll = list()
            for ii in range(len(content)):
                if content.eq(ii).attr.title:
                    ll.append((content.eq(ii).attr.title,content.eq(ii).attr.href))                    
            return ll
    return None    

In [7]:
disurls = list(pmw.find_all(distable,fieldlist=['url']))
disurls[:2]

[{'url': 'http://jbk.39.net/ptt/'}, {'url': 'http://jbk.39.net/pjs/'}]

In [8]:
zzurlset = set()
disurllist = list()#已处理url
errdisurllist = list()
for i in disurls:
    try:
        ll = parsezzurls(i['url'])
        disurllist.append(i)
        if ll:
            zzurlset = zzurlset.union(ll)
    except Exception as e:
        print(e)
        print(i)
        errdisurllist.append(i)

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
{'url': 'http://jbk.39.net/fzfxdxxdgs/'}
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
{'url': 'http://jbk.39.net/ysb3/'}
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
{'url': 'http://jbk.39.net/hcxjxyxy/'}
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
{'url': 'http://jbk.39.net/xemxyxy/'}
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
{'url': 'http://jbk.39.net/yxfl/'}
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
{'url': 'http://jbk.39.net/yxrl/'}
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
{'url': 'http://jbk.39.net/etyxgnbqbzxlxbjszzhz/'}
('Connection aborted.', RemoteDisconnected('Remote end closed connec

In [16]:
len(errdisurllist),len(disurllist)

(55, 7395)

In [17]:
zzurlset = set()
for i in errdisurllist:
    try:
        ll = parsezzurls(i['url'])
        if ll:
            zzurlset = zzurlset.union(ll)
    except Exception as e:
        print(e)
        print(i)

In [18]:
len(zzurlset)

133

In [19]:
%%time
#爬取zz详情
errorlist1 = list()
for i in list(zzurlset):
    if list(pmw.find_all(zztable,{'url':i[1]})):
        continue
    try:
        d = parse_zz(i[1])
        zztable.insert_one(d)
        print('{} is inserted!'.format(i[0]))
    except Exception as e:
        print(e)
        print(i)
        errorlist1.append(i)
    

Document is empty
('眼睑下垂', 'http://jbk.39.net/zhengzhuang/yjxc/')
CPU times: user 56.4 ms, sys: 11.2 ms, total: 67.7 ms
Wall time: 222 ms


In [49]:
parsezzurls('http://jbk.39.net/ptt/')

[('单侧头痛', 'http://jbk.39.net/zhengzhuang/dctt/'),
 ('电击样痛或刺痛的头痛', 'http://jbk.39.net/zhengzhuang/djythctdtt/'),
 ('头痛为搏动性', 'http://jbk.39.net/zhengzhuang/ttwbdx/')]

In [51]:
('单侧头痛', 'http://jbk.39.net/zhengzhuang/dctt/') == (' 单侧头痛', 'http://jbk.39.net/zhengzhuang/dctt/')

False